In [1]:
import warnings
import pandas as pd
import numpy as np

warnings.filterwarnings('ignore')
pd.__version__

'1.1.0'

# 2 基于query()的高效查询

In [2]:
# 读入netflix发行电影与剧集数据
netflix = pd.read_csv('netflix_titles.csv').fillna('')
netflix.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


In [3]:
# 找出类型为TV Show且国家不含美国的Kids' TV记录

# 传统做法
result1 = netflix[(netflix['type'] == 'TV Show')&(~netflix['country'].str.contains('United States'))&(netflix['listed_in'] == 'Kids\' TV')]

# 基于query()的做法
result2 = netflix.query("type == 'TV Show' & ~country.str.contains('United States') & listed_in == \"Kids' TV\"", 
                        engine='python')

# 检查两种方式计算出来的结果是否完全相等
(result1 == result2).all().all()

True

## 2.1 直接解析字段名

In [4]:
netflix['中文字段'] = ''
netflix['中 文 字 段'] = ''

print(netflix.query('中文字段 == ""').shape)

(6234, 14)


In [5]:
try:
    print(netflix.query('中 文 字 段 == ""').shape)
except SyntaxError:
    print('不添加`时出现错误！')
    print(netflix.query('`中 文 字 段` == ""').shape)

不添加`时出现错误！
(6234, 14)


## 2.2 链式表达式

In [6]:
demo = pd.DataFrame({
    'a': [5, 4, 3, 2, 1],
    'b': [1, 2, 3, 4, 5]
})

demo.query("a <= b != 4")

,a,b
2,3,3
4,1,5


## 2.3 支持in判断

In [7]:
netflix.query("release_year in [2018, 2019]")

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,中文字段,中 文 字 段
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...,,
27,80203094,Movie,City of Joy,Madeleine Gavin,,"United States,","September 7, 2018",2018,TV-MA,77 min,Documentaries,Women who've been sexually brutalized in war-t...,,
28,80190843,TV Show,First and Last,,,,"September 7, 2018",2018,TV-MA,1 Season,Docuseries,Take an intimate look at the emotionally charg...,,
30,80988892,Movie,Next Gen,"Kevin R. Adams, Joe Ksander","John Krasinski, Charlyne Yi, Jason Sudeikis, M...","China, Canada, United States","September 7, 2018",2018,TV-PG,106 min,"Children & Family Movies, Comedies, Sci-Fi & F...",When lonely Mai forms an unlikely bond with a ...,,
31,80239639,Movie,Sierra Burgess Is A Loser,Ian Samuels,"Shannon Purser, Kristine Froseth, RJ Cyler, No...",United States,"September 7, 2018",2018,PG-13,106 min,"Comedies, Romantic Movies",A wrong-number text sparks a virtual romance b...,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6206,80173174,TV Show,Samantha!,,"Emmanuelle Araújo, Douglas Silva, Sabrina Nona...",Brazil,"April 19, 2019",2019,TV-MA,2 Seasons,"International TV Shows, TV Comedies","A child star in the '80s, Samantha clings to t...",,
6215,80066227,TV Show,Crazy Ex-Girlfriend,,"Rachel Bloom, Vincent Rodriguez III, Santino F...",United States,"April 13, 2019",2019,TV-14,4 Seasons,"Romantic TV Shows, TV Comedies","Still pining for Josh, the boy who dumped her ...",,
6216,80108373,TV Show,The Magic School Bus Rides Again,,"Kate McKinnon, Miles Koseleci-Vieira, Mikaela ...",United States,"April 13, 2018",2018,TV-Y,2 Seasons,Kids' TV,Ms. Frizzle's kid sister Fiona takes the wheel...,,
6219,80186475,TV Show,Pokémon the Series,,"Sarah Natochenny, Laurie Hymes, Jessica Paquet...",Japan,"April 1, 2019",2019,TV-Y7-FV,2 Seasons,"Anime Series, Kids' TV",Ash and his Pikachu travel to the Alola region...,,


## 2.4 对外部变量的支持

In [8]:
years = [2018, 2019]
netflix.query("release_year in @years")

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,中文字段,中 文 字 段
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...,,
27,80203094,Movie,City of Joy,Madeleine Gavin,,"United States,","September 7, 2018",2018,TV-MA,77 min,Documentaries,Women who've been sexually brutalized in war-t...,,
28,80190843,TV Show,First and Last,,,,"September 7, 2018",2018,TV-MA,1 Season,Docuseries,Take an intimate look at the emotionally charg...,,
30,80988892,Movie,Next Gen,"Kevin R. Adams, Joe Ksander","John Krasinski, Charlyne Yi, Jason Sudeikis, M...","China, Canada, United States","September 7, 2018",2018,TV-PG,106 min,"Children & Family Movies, Comedies, Sci-Fi & F...",When lonely Mai forms an unlikely bond with a ...,,
31,80239639,Movie,Sierra Burgess Is A Loser,Ian Samuels,"Shannon Purser, Kristine Froseth, RJ Cyler, No...",United States,"September 7, 2018",2018,PG-13,106 min,"Comedies, Romantic Movies",A wrong-number text sparks a virtual romance b...,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6206,80173174,TV Show,Samantha!,,"Emmanuelle Araújo, Douglas Silva, Sabrina Nona...",Brazil,"April 19, 2019",2019,TV-MA,2 Seasons,"International TV Shows, TV Comedies","A child star in the '80s, Samantha clings to t...",,
6215,80066227,TV Show,Crazy Ex-Girlfriend,,"Rachel Bloom, Vincent Rodriguez III, Santino F...",United States,"April 13, 2019",2019,TV-14,4 Seasons,"Romantic TV Shows, TV Comedies","Still pining for Josh, the boy who dumped her ...",,
6216,80108373,TV Show,The Magic School Bus Rides Again,,"Kate McKinnon, Miles Koseleci-Vieira, Mikaela ...",United States,"April 13, 2018",2018,TV-Y,2 Seasons,Kids' TV,Ms. Frizzle's kid sister Fiona takes the wheel...,,
6219,80186475,TV Show,Pokémon the Series,,"Sarah Natochenny, Laurie Hymes, Jessica Paquet...",Japan,"April 1, 2019",2019,TV-Y7-FV,2 Seasons,"Anime Series, Kids' TV",Ash and his Pikachu travel to the Alola region...,,


## 2.5 对常规语句的支持

In [9]:
def country_count(s):
    '''
    计算涉及国家数量
    '''
    
    return s.split(',').__len__()

# 找出发行年份在2018或2019年且合作国家数量超过5个的剧集
netflix.query("release_year.isin([2018, 2019]) and country.apply(@country_count) > 5")

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,中文字段,中 文 字 段
6139,80095299,TV Show,Ultimate Beastmaster,,"Tiki Barber, Chris Distefano, Francesco Facchi...","United States, Brazil, South Korea, Mexico, Ja...","August 31, 2018",2018,TV-14,3 Seasons,Reality TV,"In this intense obstacle course series, elite ...",,
6140,80156006,TV Show,Ultimate Beastmaster México,,"Luis Ernesto Franco, Inés Sainz, Tiki Barber, ...","United States, Brazil, South Korea, Mexico, Ja...","August 31, 2018",2018,TV-14,3 Seasons,"Reality TV, Spanish-Language TV Shows",Competitors representing countries from around...,,


## 2.6 对Index与MultiIndex的支持

- **常规index**

In [10]:
# 找出索引列中包含king的记录，忽略大小写
netflix.set_index('title').query("index.str.contains('king', case=False)")

,show_id,type,director,cast,country,date_added,release_year,rating,duration,listed_in,description,中文字段,中 文 字 段
title,,,,,,,,,,,,,
Norm of the North: King Sized Adventure,81145628,Movie,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...,,
The Sinking Of Van Der Wijck,81002866,Movie,Sunil Soraya,"Herjunot Ali, Pevita Pearce, Reza Rahadian, Ra...",Indonesia,"September 28, 2018",2013,TV-14,163 min,"Dramas, International Movies, Romantic Movies",When a young man leaves home to fulfill the wi...,,
Joseph: King of Dreams,60003155,Movie,"Rob LaDuca, Robert C. Ramirez","Ben Affleck, Mark Hamill, Richard Herd, Mauree...",United States,"September 27, 2011",2000,TV-PG,75 min,"Children & Family Movies, Dramas, Faith & Spir...",With his gift of dream interpretation and his ...,,
King of Boys,81172721,Movie,Kemi Adetiba,"Sola Sobowale, Adesua Etomi, Remilekun Safaru,...",Nigeria,"September 21, 2019",2018,TV-MA,182 min,"Dramas, International Movies",When a powerful businesswoman’s political ambi...,,
The Crystal Calls Making the Dark Crystal: Age of Resistance,80238013,Movie,Randall Lobb,"Taron Egerton, Natalie Dormer, Simon Pegg, Jas...",United States,"September 14, 2019",2019,TV-PG,83 min,"Documentaries, International Movies","Go behind the scenes with stars, puppeteers an...",,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
All Hail King Julien,80018987,TV Show,,"Danny Jacobs, Andy Richter, Henry Winkler, Kev...",United States,"December 1, 2017",2017,TV-Y7-FV,5 Seasons,"Kids' TV, TV Comedies",In this Emmy winner for Outstanding Children's...,,
The Great British Baking Show,80063224,TV Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,"August 30, 2019",2019,TV-PG,7 Seasons,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...,,
True and the Rainbow Kingdom,80063599,TV Show,,"Michela Luci, Jamie Watson, Anna Claire Bartla...",Canada,"August 30, 2019",2019,TV-Y,3 Seasons,Kids' TV,"When something's wrong in the Rainbow Kingdom,...",,


- **names为空的MultiIndex**

In [11]:
# 构造含有MultiIndex的数据框，并重置index的names为None
temp = netflix.set_index(['title', 'type']);temp.index.names = (None, None)

# 找出第一个index包含king（忽略大小写），第二个index等于Movie的记录
temp.query("ilevel_0.str.contains('king', case=False) and ilevel_1 == 'Movie'")

,,show_id,director,cast,country,date_added,release_year,rating,duration,listed_in,description,中文字段,中 文 字 段
Norm of the North: King Sized Adventure,Movie,81145628,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...,,
The Sinking Of Van Der Wijck,Movie,81002866,Sunil Soraya,"Herjunot Ali, Pevita Pearce, Reza Rahadian, Ra...",Indonesia,"September 28, 2018",2013,TV-14,163 min,"Dramas, International Movies, Romantic Movies",When a young man leaves home to fulfill the wi...,,
Joseph: King of Dreams,Movie,60003155,"Rob LaDuca, Robert C. Ramirez","Ben Affleck, Mark Hamill, Richard Herd, Mauree...",United States,"September 27, 2011",2000,TV-PG,75 min,"Children & Family Movies, Dramas, Faith & Spir...",With his gift of dream interpretation and his ...,,
King of Boys,Movie,81172721,Kemi Adetiba,"Sola Sobowale, Adesua Etomi, Remilekun Safaru,...",Nigeria,"September 21, 2019",2018,TV-MA,182 min,"Dramas, International Movies",When a powerful businesswoman’s political ambi...,,
The Crystal Calls Making the Dark Crystal: Age of Resistance,Movie,80238013,Randall Lobb,"Taron Egerton, Natalie Dormer, Simon Pegg, Jas...",United States,"September 14, 2019",2019,TV-PG,83 min,"Documentaries, International Movies","Go behind the scenes with stars, puppeteers an...",,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Jeff Foxworthy and Larry the Cable Guy: We’ve Been Thinking...,Movie,80097431,Jay Karas,"Jeff Foxworthy, Larry the Cable Guy",United States,"August 26, 2016",2016,TV-14,75 min,Stand-Up Comedy,Grammy-nominated comedians Jeff Foxworthy and ...,,
Jim Gaffigan: King Baby,Movie,70113636,Troy Miller,Jim Gaffigan,United States,"August 11, 2016",2009,TV-PG,71 min,Stand-Up Comedy,"Jim Gaffigan offers up his take on bacon, bowl...",,
Alive and Kicking,Movie,80105356,Susan Glatzer,,"Sweden, United States","August 10, 2017",2016,TV-14,89 min,Documentaries,Take an inside look at swing dancing's continu...,,
Rodney King,Movie,80128715,Spike Lee,Roger Guenveur Smith,United States,"April 28, 2017",2017,TV-MA,53 min,Dramas,Roger Guenveur Smith gives voice to the man at...,,


- **names不为空的MultiIndex**

In [12]:
# 构造含有MultiIndex的数据框，并重置index的names为None
temp = netflix.set_index(['title', 'type'])

# 找出第一个index包含king（忽略大小写），第二个index等于Movie的记录
temp.query("title.str.contains('king', case=False) and type == 'Movie'")

,,show_id,director,cast,country,date_added,release_year,rating,duration,listed_in,description,中文字段,中 文 字 段
title,type,,,,,,,,,,,,
Norm of the North: King Sized Adventure,Movie,81145628,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...,,
The Sinking Of Van Der Wijck,Movie,81002866,Sunil Soraya,"Herjunot Ali, Pevita Pearce, Reza Rahadian, Ra...",Indonesia,"September 28, 2018",2013,TV-14,163 min,"Dramas, International Movies, Romantic Movies",When a young man leaves home to fulfill the wi...,,
Joseph: King of Dreams,Movie,60003155,"Rob LaDuca, Robert C. Ramirez","Ben Affleck, Mark Hamill, Richard Herd, Mauree...",United States,"September 27, 2011",2000,TV-PG,75 min,"Children & Family Movies, Dramas, Faith & Spir...",With his gift of dream interpretation and his ...,,
King of Boys,Movie,81172721,Kemi Adetiba,"Sola Sobowale, Adesua Etomi, Remilekun Safaru,...",Nigeria,"September 21, 2019",2018,TV-MA,182 min,"Dramas, International Movies",When a powerful businesswoman’s political ambi...,,
The Crystal Calls Making the Dark Crystal: Age of Resistance,Movie,80238013,Randall Lobb,"Taron Egerton, Natalie Dormer, Simon Pegg, Jas...",United States,"September 14, 2019",2019,TV-PG,83 min,"Documentaries, International Movies","Go behind the scenes with stars, puppeteers an...",,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Jeff Foxworthy and Larry the Cable Guy: We’ve Been Thinking...,Movie,80097431,Jay Karas,"Jeff Foxworthy, Larry the Cable Guy",United States,"August 26, 2016",2016,TV-14,75 min,Stand-Up Comedy,Grammy-nominated comedians Jeff Foxworthy and ...,,
Jim Gaffigan: King Baby,Movie,70113636,Troy Miller,Jim Gaffigan,United States,"August 11, 2016",2009,TV-PG,71 min,Stand-Up Comedy,"Jim Gaffigan offers up his take on bacon, bowl...",,
Alive and Kicking,Movie,80105356,Susan Glatzer,,"Sweden, United States","August 10, 2017",2016,TV-14,89 min,Documentaries,Take an inside look at swing dancing's continu...,,


# 3 基于eval()的高效运算

In [13]:
netflix.drop(columns=['中文字段', '中 文 字 段'], inplace=True) # 删除无关字段

In [14]:
netflix['date_added'][0]

'September 9, 2019'

In [15]:
# 利用assign进行新增字段计算并保存为新数据框
result1 = netflix.assign(years_to_now=2020 - netflix['release_year'],
                         new_date_added=pd.to_datetime(netflix['date_added'].str.strip(), 
                                                       format='%B %d, %Y', errors='coerce'))

# 利用eval()进行新增字段计算并保存为新数据框
result2 = netflix.eval('''
                       years_to_now = 2020 - release_year
                       new_date_added = @pd.to_datetime(date_added.str.strip(), format='%B %d, %Y', errors='coerce')''')

(result1 == result2).all()

show_id            True
type               True
title              True
director           True
cast               True
country            True
date_added         True
release_year       True
rating             True
duration           True
listed_in          True
description        True
years_to_now       True
new_date_added    False
dtype: bool

In [16]:
netflix.eval('''
             years_to_now = 2020 - release_year
             new_date_added = @pd.to_datetime(date_added.str.strip(), 
                                              format='%B %d, %Y', 
                                              errors='coerce')''')

TokenError: ('EOF in multi-line statement', (2, 0))

In [17]:
from functools import partial

# 利用partial固化指定参数
func = partial(pd.to_datetime, format='%B %d, %Y', errors='coerce')

netflix.eval('''
             years_to_now = 2020 - release_year
             new_date_added = @func(date_added.str.strip())''')

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,years_to_now,new_date_added
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...,1,2019-09-09
1,80117401,Movie,Jandino: Whatever it Takes,,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...,4,2016-09-09
2,70234439,TV Show,Transformers Prime,,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob...",7,2018-09-08
3,80058654,TV Show,Transformers: Robots in Disguise,,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...,4,2018-09-08
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...,3,2017-09-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6229,80000063,TV Show,Red vs. Blue,,"Burnie Burns, Jason Saldaña, Gustavo Sorola, G...",United States,,2015,NR,13 Seasons,"TV Action & Adventure, TV Comedies, TV Sci-Fi ...","This parody of first-person shooter games, mil...",5,NaT
6230,70286564,TV Show,Maron,,"Marc Maron, Judd Hirsch, Josh Brener, Nora Zeh...",United States,,2016,TV-MA,4 Seasons,TV Comedies,"Marc Maron stars as Marc Maron, who interviews...",4,NaT
6231,80116008,Movie,Little Baby Bum: Nursery Rhyme Friends,,,,,2016,,60 min,Movies,Nursery rhymes and original music for children...,4,NaT
6232,70281022,TV Show,A Young Doctor's Notebook and Other Stories,,"Daniel Radcliffe, Jon Hamm, Adam Godley, Chris...",United Kingdom,,2013,TV-MA,2 Seasons,"British TV Shows, TV Comedies, TV Dramas","Set during the Russian Revolution, this comic ...",7,NaT


In [18]:
netflix.eval('''
             years_to_now = 2020 - release_year
             new_date_added = @func(date_added.str.strip())''') \
       .resample('M', on='new_date_added') \
       .agg({'new_date_added': 'count'}) \
       .rename(columns={'new_date_added': '月度发行数量'}) \
       .eval('''月度发行数量排名 = 月度发行数量.rank(ascending=False).astype('int')''') \
       .sort_values('月度发行数量排名')

,月度发行数量,月度发行数量排名
new_date_added,,
2019-11-30,292,1
2019-12-31,232,2
2019-10-31,220,3
2018-12-31,203,4
2018-10-31,201,5
...,...,...
2010-08-31,0,119
2010-07-31,0,119
2010-05-31,0,119
